In [10]:
import numpy as np
import pandas as pd
import os,sys
import cv2
import pickle

In [11]:
# initialize the list of points for the rectangle bbox,
# the temporaray endpoint of the drawing rectangle
# the list of all bounding boxes of selected rois
# and boolean indicating wether drawing of mouse
# is performed or not
rect_endpoint_tmp = []
rect_bbox = []

drawing = False

def check_boxes(img_clean,bbox_list):
    def draw_all_boxes():
        img = img_clean.copy()
        
        for b in bbox_list:
            cv2.rectangle(img, (b[0],b[1]),(b[2],b[3]), color=(0, 255, 0),thickness=2)
        cv2.imshow('image', img)
            
        
    # mouse callback function
    def draw_rect_roi(event, x, y, flags, param):
            # grab references to the global variables
            global rect_bbox, rect_endpoint_tmp, drawing
    
            # if the left mouse button was clicked, record the starting
            # (x, y) coordinates and indicate that drawing is being
            # performed. set rect_endpoint_tmp empty list.
            if event == cv2.EVENT_LBUTTONDOWN:
                rect_endpoint_tmp = []
                rect_bbox = [(x, y)]
                drawing = True
    
            # check to see if the left mouse button was released
            elif event == cv2.EVENT_LBUTTONUP:
                # record the ending (x, y) coordinates and indicate that
                # drawing operation is finished
                rect_bbox.append((x, y))
                drawing = False
    
                # draw a rectangle around the region of interest
                p_1, p_2 = rect_bbox
                
                # for bbox find upper left and bottom right points
                p_1x, p_1y = p_1
                p_2x, p_2y = p_2
    
                lx = min(p_1x, p_2x)
                ty = min(p_1y, p_2y)
                rx = max(p_1x, p_2x)
                by = max(p_1y, p_2y)
    
                # add bbox to list if both points are different
                if (lx, ty) != (rx, by):
                    if abs(lx-rx)>5:
                        if abs(ty-by)>5:
                            bbox = [lx, ty, rx, by]
                            bbox_list.append(bbox)
    
            # if mouse is drawing set tmp rectangle endpoint to (x,y)
            elif event == cv2.EVENT_MOUSEMOVE and drawing:
                rect_endpoint_tmp = [(x, y)]
            elif event == cv2.EVENT_LBUTTONDBLCLK:
                npbx=np.asarray(bbox_list)
                selected_box = ((x>npbx[:,0]) & (y>npbx[:,1]) & (x<npbx[:,2]) & (y<npbx[:,3]))
                if np.sum(selected_box)==1:
                    bbox_list.remove(npbx[selected_box].tolist()[0])
                if np.sum(selected_box)>1:
                    potentials = npbx[selected_box]
                    areas = (potentials[:,2]-potentials[:,0])*(potentials[:,3]-potentials[:,1])
                    bbox_list.remove(potentials[np.argmin(areas)].tolist())
                draw_all_boxes()

    cv2.namedWindow('image',cv2.WINDOW_GUI_NORMAL )
    cv2.resizeWindow('image', 800,800)
    cv2.setMouseCallback('image', draw_rect_roi)
    draw_all_boxes()
    # keep looping until the 'c' key is pressed
    stop = False
    while True:
        # display the image and wait for a keypress
        if not drawing:
            draw_all_boxes()
            #cv2.imshow('image', img)
        elif drawing and rect_endpoint_tmp:
            rect_cpy = img_clean.copy()
            start_point = rect_bbox[0]
            end_point_tmp = rect_endpoint_tmp[0]
            cv2.rectangle(rect_cpy, start_point, end_point_tmp,(0,255,0),2)
            cv2.imshow('image', rect_cpy)
    
        key = cv2.waitKey(1) #& 0xFF
        # if the 'c' key is pressed, break from the loop
        if key == ord('c'):
            break
        if key == ord('q'):
            stop=True
            break
    # close all open windows
    cv2.destroyAllWindows()
    #cv2.waitKey(1)
    return stop

In [12]:

im_dir = '/home/ctorney/euclid/workspace/uavTracker/train/horse_images/'

with open (im_dir + 'annotations-trained.pickle', 'rb') as fp:
    all_imgs = pickle.load(fp)



from_scratch=False
if from_scratch:
    new_imgs = []
else:
    with open (im_dir + 'annotations-checked.pickle', 'rb') as fp:
        new_imgs = pickle.load(fp)

for i in range(len(all_imgs)):
    basename = os.path.basename(all_imgs[i]['filename'])
    if not from_scratch:
        if any(d['filename'] == basename for d in new_imgs):
            continue
    img_data = {'object':[]}
    img_data['filename'] = basename
    img_data['width'] = all_imgs[i]['width']
    img_data['height'] = all_imgs[i]['height']
    sys.stdout.write(img_data['filename'] + ", " + str(i) + ' of ' + str(len(all_imgs)) + " \n=====================================\n" )
    sys.stdout.flush()
    
    boxes=[]
    for obj in all_imgs[i]['object']:
        boxes.append([obj['xmin'],obj['ymin'],obj['xmax'],obj['ymax']])

    #do box processing
    img = cv2.imread(im_dir + basename)
    if check_boxes(img,boxes):
        break
    for b in boxes:
        obj = {}
        if ((b[2]-b[0])*(b[3]-b[1]))<10:
            continue
        obj['name'] = 'aoi'
        obj['xmin'] = int(b[0])
        obj['ymin'] = int(b[1])
        obj['xmax'] = int(b[2])
        obj['ymax'] = int(b[3])
        img_data['object'] += [obj]

    new_imgs += [img_data]

#print(all_imgs)
with open(im_dir + 'annotations-checked.pickle', 'wb') as handle:
   pickle.dump(new_imgs, handle)

TR_180523-4-4521_half-0.png, 408 of 2000 
TR_180523-4-4521_half-1.png, 409 of 2000 
TR_180616-14-4297_half-0.png, 410 of 2000 
TR_180616-14-4297_half-1.png, 411 of 2000 
TR_180618-16-1348_half-0.png, 412 of 2000 
TR_180618-16-1348_half-1.png, 413 of 2000 
TR_180616-7-5829_half-0.png, 414 of 2000 
TR_180616-7-5829_half-1.png, 415 of 2000 
TR_170704-5-1-1728_half-0.png, 416 of 2000 
TR_170704-5-1-1728_half-1.png, 417 of 2000 
TR_170623-4-2-5458_half-0.png, 418 of 2000 
TR_170623-4-2-5458_half-1.png, 419 of 2000 
TR_180620-11-10742_half-0.png, 420 of 2000 
TR_180620-11-10742_half-1.png, 421 of 2000 
TR_180621-15-4643_half-0.png, 422 of 2000 
TR_180621-15-4643_half-1.png, 423 of 2000 
TR_170613-2-1-1201_half-0.png, 424 of 2000 
TR_170613-2-1-1201_half-1.png, 425 of 2000 
TR_180602-14-8778_half-0.png, 426 of 2000 
TR_180602-14-8778_half-1.png, 427 of 2000 
TR_170703-1-2-455_half-0.png, 428 of 2000 
TR_170703-1-2-455_half-1.png, 429 of 2000 
TR_180620-7-3149_half-0.png, 430 of 2000 
TR_18062

TR_170606-10-221_half-1.png, 509 of 2000 
TR_170711-1-1-14039_half-0.png, 510 of 2000 
TR_170711-1-1-14039_half-1.png, 511 of 2000 
TR_180618-2-5550_half-0.png, 512 of 2000 
TR_180618-2-5550_half-1.png, 513 of 2000 
TR_180627-2-2917_half-0.png, 514 of 2000 
TR_180627-2-2917_half-1.png, 515 of 2000 
TR_180627-7-13314_half-0.png, 516 of 2000 
TR_180627-7-13314_half-1.png, 517 of 2000 
TR_DJI_0001-11650_half-0.png, 518 of 2000 
TR_DJI_0001-11650_half-1.png, 519 of 2000 
TR_180621-14-6389_half-0.png, 520 of 2000 
TR_180621-14-6389_half-1.png, 521 of 2000 
TR_180627-17-16216_half-0.png, 522 of 2000 
TR_180627-17-16216_half-1.png, 523 of 2000 
TR_170607-1-1922_half-0.png, 524 of 2000 
TR_170607-1-1922_half-1.png, 525 of 2000 
TR_170713-8-1-3694_half-0.png, 526 of 2000 
TR_170713-8-1-3694_half-1.png, 527 of 2000 
TR_180517-5-2280_half-0.png, 528 of 2000 
TR_180517-5-2280_half-1.png, 529 of 2000 
TR_170608-3-3390_half-0.png, 530 of 2000 
TR_170608-3-3390_half-1.png, 531 of 2000 
TR_170606-4-31